# Import data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

import os

DATA_DIR = os.path.join(
    path,
    "asl_alphabet_train",
    "asl_alphabet_train"
)


# Import lib

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Chuan hoa du lieu

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 128

In [ ]:
# cắt tập train : validation + scale ảnh
datagen = ImageDataGenerator(
    rescale=1./255, # chuyen doi gia tri [0, 255] ve [0, 1]
    validation_split=0.2 # chia du lieu thanh 80% train va 20% validation
)

In [ ]:
train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # chi dinh lay tap train
)

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # chi dinh lay tap validation
)

# Khai bao mo hinh - fit

In [ ]:
model = Sequential(
    [
        Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2,2)),
        
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        
        Flatten(),
        Dense(512, activation='relu'), # fully connected layer
        Dropout(0.5),  # dropout de tranh overfitting
        Dense(train_data.num_classes, activation='softmax') # 29 classes trong ASL alphabet
    ]
)

model.summary()

In [26]:
# huan luyen
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

544/544 ━━━━━━━━━━━━━━━━━━━━ 120s 217ms/step - accuracy: 0.6513 - loss: 1.1467 - val_accuracy: 0.6860 - val_loss: 1.0465
Epoch 2/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 110s 201ms/step - accuracy: 0.9392 - loss: 0.1859 - val_accuracy: 0.7367 - val_loss: 1.1271
Epoch 3/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 109s 200ms/step - accuracy: 0.9707 - loss: 0.0895 - val_accuracy: 0.7598 - val_loss: 1.0887
Epoch 4/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 140s 257ms/step - accuracy: 0.9828 - loss: 0.0537 - val_accuracy: 0.7618 - val_loss: 1.2040
Epoch 5/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 124s 227ms/step - accuracy: 0.9854 - loss: 0.0450 - val_accuracy: 0.7711 - val_loss: 1.0802
Epoch 6/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 126s 231ms/step - accuracy: 0.9879 - loss: 0.0393 - val_accuracy: 0.7295 - val_loss: 1.3172
Epoch 7/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 141s 259ms/step - accuracy: 0.9905 - loss: 0.0310 - val_accuracy: 0.7687 - val_loss: 1.3187
Epoch 8/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 124s 228ms/step - accuracy: 0.9924 - loss: 0.02

# Save model

In [ ]:
model.save('asl_alphabet_model.h5') # .h5: dung de load lai va du doan cho sau nay
print('Saved model')

AttributeError: module 'keras._tf_keras.keras' has no attribute 'saving'

# Danh gia mo hinh

In [28]:
loss, accuracy = model.evaluate(val_data)
print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {loss:.4f}')

136/136 ━━━━━━━━━━━━━━━━━━━━ 16s 117ms/step - accuracy: 0.7625 - loss: 1.2635
Accuracy: 0.7625
Loss: 1.2635


# Du doan

In [29]:
# load mo hinh 
loaded_model = keras.models.load_model('asl_alphabet_model.h5')
print('Loaded model from disk')

Loaded model from disk


In [30]:
# chuan bi label (chu cai)
class_names = list(train_data.class_indices.keys())
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [34]:
import os
# load 1 hinh de xu ly
img_dir = os.path.join(path, "asl_alphabet_train", "asl_alphabet_train", "M")  # duong dan den thu muc chua hinh
# Get first image file
img_file = os.listdir(img_dir)[0]
img_path = os.path.join(img_dir, img_file) # duong dan den hinh can du doan
# img_path = input("Enter the path of the image to predict: ")
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0)  # them kich thuoc

In [35]:
# du doan
prediction = loaded_model.predict(img_array)
predicted_index = np.argmax(prediction)
predicted_label = class_names[predicted_index] # map index -> label

# print % cac lop du doan
print(prediction)
print(f'Predicted label: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[4.9077283e-18 4.4073430e-24 3.1672995e-35 7.8456207e-38 2.6034555e-17
  9.9570848e-27 1.3169288e-16 2.8684500e-23 2.1881641e-21 2.8176604e-13
  4.4430905e-19 1.0377214e-19 1.0000000e+00 4.1443796e-11 6.2759394e-23
  4.3143451e-21 4.4553306e-25 2.3015677e-17 2.9727523e-13 1.4828903e-16
  1.1010739e-13 2.2052127e-17 2.5302700e-22 1.8775224e-12 5.7355473e-19
  1.3584987e-25 1.4305641e-18 4.0314793e-25 5.8764986e-24]]
Predicted label: M
